In [ ]:
import os
import numpy as np
from PIL import Image

In [7]:
import pandas as pd
import os

In [11]:
DATA_DIR = "/kaggle/input/UBC-OCEAN"
OUTPUT_DIR = "/kaggle/working/ubc-ocean-tiles"
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
TRAIN_IMGS = os.path.join(DATA_DIR, "train_images")

In [12]:
data_df = pd.read_csv(TRAIN_CSV) 
data_df["path"] = [os.path.join(TRAIN_IMGS, f"{str(i)}.png")
                   for i in data_df["image_id"]]


In [ ]:
# roubar as coisas do agrotech
# fazer o split para 2048x2048
# resize para 256x256

In [16]:
def extract_image_tiles(p_img, out_folder, size: int = 768, scale: float = 1.0, drop_thr: float = 0.8) -> list:
    name, _ = os.path.splitext(os.path.basename(p_img))
    im = pyvips.Image.new_from_file(p_img)
    w = h = size
    # https://stackoverflow.com/a/47581978/4521646
    idxs = [(y, y + h, x, x + w) for y in range(0, im.height, h) for x in range(0, im.width, w)]
    files = []
    for k, (y, y_, x, x_) in enumerate(idxs):
        # https://libvips.github.io/pyvips/vimage.html#pyvips.Image.crop
        tile = im.crop(x, y, min(w, im.width - x), min(h, im.height - y)).numpy()[..., :3]
        if tile.shape[:2] != (h, w):
            tile_ = tile
            tile_size = (h, w) if tile.ndim == 2 else (h, w, tile.shape[2])
            tile = np.zeros(tile_size, dtype=tile.dtype)
            tile[:tile_.shape[0], :tile_.shape[1], ...] = tile_
        mask_bg = np.sum(tile, axis=2) == 0
        if np.sum(mask_bg) >= (np.prod(mask_bg.shape) * drop_thr):
            #print(f"skip almost empty tile: {k:06}_{int(x_ / w)}-{int(y_ / h)}")
            continue
        tile[mask_bg, :] = 255
        mask_bg = np.mean(tile, axis=2) > 240
        if np.sum(mask_bg) >= (np.prod(mask_bg.shape) * drop_thr):
            #print(f"skip almost empty tile: {k:06}_{int(x_ / w)}-{int(y_ / h)}")
            continue
        p_img = os.path.join(folder, f"{k:06}_{int(x_ / w)}-{int(y_ / h)}.png")
        # print(tile.shape, tile.dtype, tile.min(), tile.max())
        new_size = int(size * scale), int(size * scale)
        Image.fromarray(tile).resize(new_size, Image.LANCZOS).save(p_img)
        files.append(p_img)
    return files, idxs

0       9.5140
1      19.5484
2       1.3552
3      16.9236
4      14.8816
        ...   
533    21.3420
534    22.0168
535    30.3440
536    17.0204
537    18.0760
Name: image_width, Length: 538, dtype: float64

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session